In [1]:
import numpy as np

In [2]:
# 定义变量
m = 1000 
theta = 0.75
alp = 1.0

In [3]:
# 定义文件夹路径
folder = '/home/louyaxin/1D-Hydrodynamic-Model/input/'
folder2 = 'result/'

In [4]:
# 打开拓扑文件
with open(folder + 'TOPO.txt') as f:
    JuncNum, JuncNum1, JuncNum2 = [int(x) for x in f.readline().split()] 
    ChanNum, chanNum1, chanNum2 = [int(x) for x in f.readline().split()]
    UbouNum, AbouNum = [int(x) for x in f.readline().split()]

In [5]:
print(f'ChanNum={ChanNum}, chanNum1={chanNum1}, chanNum2={chanNum2}') 
print(f'JuncNum={JuncNum}, JuncNum1={JuncNum1}, JuncNum2={JuncNum2}')
print('river star end')

ChanNum=3, chanNum1=0, chanNum2=1
JuncNum=4, JuncNum1=1, JuncNum2=2
river star end


In [6]:
#打开时间步长文件
with open(folder + 'TIME.txt') as f:
    dt = float(f.readline())  
    rdt = float(f.readline())

In [7]:
def read_file(f):
    chan = [] 
    for i in range(ChanNum):
        no, st, en, name = f.readline().split()
        chan.append({'no': int(no), 'st': int(st), 'en': int(en), 'name': name})

# 分类节点
Junc = np.zeros(JuncNum, dtype=[('no', 'int'), ('flag', 'int')])
for i in range(JuncNum1+1, JuncNum2):
    Junc[i]['flag'] = 1
    print(f'Nodes {i} is downstream boundary point')
    
for i in range(JuncNum2+1, JuncNum):
    Junc[i]['flag'] = -1
    print(f'Nodes {i} is upstream boundary point')

Nodes 3 is upstream boundary point
